### Base line modeling

전체적인 과정은 아래와 같습니다.

|1|2|3|4|5|6|
|---|---|---|---|---|---|
|데이터 <br> 불러오기|(기본적인)<br>피처<br>엔지니어링|평가지표<br>계산 함수<br>작성|모델 훈련|성능 검증|제출|

### 데이터 불러오기

In [1]:
import pandas as pd, numpy as np

train = pd.read_csv('./data/train.csv')
test = pd.read_csv("./data/test.csv")
submission = pd.read_csv("./data/sampleSubmission.csv")

### 피처 엔지니어링
데이터를 변환하는 작업을 일컫습니다. 보통은 이 변환을 훈련 데이터와 테스트 데이터에 공통으로 반영해야 하기 때문에, 피처 엔지니어링 전에 두 데이터를 합쳤다가 끝나면 다시 분할합니다.

In [2]:
# 데이터 합치기 전, 이상치 제거하기
train = train[train['weather'] != 4]

# 데이터 병합
all_data = pd.concat([train, test], ignore_index=True)

# 파생 피처 추가
from datetime import datetime
all_data['date'] = all_data['datetime'].apply(lambda x : x.split()[0]) # 날짜 피처 생성
all_data['year'] = all_data['datetime'].apply(lambda x : x.split()[0].split('-')[0]) # 년도 피처 생성
all_data['month'] = all_data['datetime'].apply(lambda x : x.split()[0].split('-')[1]) # 년도 피처 생성
all_data['day'] = all_data['datetime'].apply(lambda x : x.split()[0].split('-')[2]) # 년도 피처 생성
all_data['weekday'] = all_data['date'].apply(lambda DateString : datetime.strptime(DateString, "%Y-%m-%d").weekday())

# 이전 데이터 분석을 토대로 필요없는 치퍼인 minute, second, day는 생성하지 않음

# 필요없는 피처 제거 : casual, registered, datetime, date, month, windspeed
drop_features = ["casual", 'registered', 'datetime', 'date', "month", "windspeed"]
all_data = all_data.drop(drop_features, axis=1)

In [3]:
# 데이터 분할하기
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# 타겟값 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)
y = train['count'] # 타겟값

# 타겟값이 있으면 훈련데이터, 없으면 테스트 데이터.
# all_data['count']가 타겟값이기 때문에 null이 아니라면 훈련데이터를 의미한다. 따라서 훈련 데이터를 추릴 때는 pd.isnull() 앞에 부정을 의미하는 ~기호를 붙였다.

X_train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,year,day,weekday
0,1,0,0,1,9.84,14.395,81,2011,01,5
1,1,0,0,1,9.02,13.635,80,2011,01,5
2,1,0,0,1,9.02,13.635,80,2011,01,5
3,1,0,0,1,9.84,14.395,75,2011,01,5
4,1,0,0,1,9.84,14.395,75,2011,01,5


### 평가지표 계산함수 작성

훈련이란 어떠한 능력을 개선하기 위해 배우거나 단련하는 행위입니다. 따라서 훈련이 제대로 이루어졌는지 확인하려면 대상 능력을 평가할 수단인 평가지표가 필요합니다.

RMSLE를 계산하는 함수를 생성합니다.

In [4]:
import numpy as np

def rmsle(y_true, y_pred, convertExp = True):
    # 지수변환
    # 지수 변환 하는 이유 : 타겟값이 정규분포를 따르지 않는다면 로그 변환을 사용해야 하는데, 원래의 값으로 되돌리려면 지수변환을 해줘야한다.
    # 타겟값이 정규분포를 따른다면 convertExp=False 인스턴스를 적어야 한다.
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
        
    # 로그변환 후 결측값을 0으로 변환
    log_true = np.nan_to_num(np.log(y_true + 1))
    log_pred = np.nan_to_num(np.log(y_pred + 1))
    
    # RMSLE 계산
    output = np.sqrt(np.mean((log_true - log_pred)**2))
    
    return output

### Model Trainning

모델 훈련 : 독립변수(feature)인 X_train과 종속변수(target)인 log_y에 대응하는 최적의 선형 회귀 계수를 구한다는 의미. 회귀식은 아래와 같다.

- Y = θ + θ<sub>1</sub> x<sub>1</sub> + θ<sub>2</sub> x<sub>2</sub> + θ<sub>3</sub> x<sub>3</sub>
    + 독립변수 x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>와 종속변수 Y를 활용하여 선형 회귀 모델을 훈련하면 독립변수와 종속변수에 대응하는 최적의 선형 회귀 계수 θ<sub>1</sub>, θ<sub>2</sub>, θ<sub>3</sub>를 구할 수 있습니다. 이 과정을 **훈련** 이라고 합니다.  θ<sub>1</sub>, θ<sub>2</sub>, θ<sub>3</sub>의 값을 아는 상태에서 새로운 독립변수 x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>가 주어진다면 종속변수 Y를 구할 수 있습니다. 이러한 과정을 **예측**이라고 합니다. 훈련 단계에서 한 번도 보지 못한 독립변수가 주어지더라도 회귀 계수를 알고 있기 때문에 종속 변수를 예측할 수 있습니다.

다시 익숙한 용어로 풀어보면 다음과 같습니다.
- 훈련 : feature(독립변수)와 타깃값(종속 변수)이 주어졌을 때 최적의 가중치(회귀 계수)를 찾는 과정
- 예측 : 최적의 가중치를 아는 상태(훈련된 모델을 바탕으로)에서 새로운 독립변수(임의의 데이터)가 주어졌을 때 타겟값을 추정하는 과정

이러한 맥락에서 탐색적 데이터 분석과 피처 엔지니어링은 다음처럼 풀어 생각할 수 있습니다.
- 탐색적 데이터 분석 : 예측에 도움이 될 피처를 추리고, 적절한 모델링 방법을 탐색하는 과정
- 피처 엔지니어링 : 추려진 피처들을 훈련에 적합하도록, 성능 향상에 도움되도록 가공하는 과정

In [5]:
from sklearn.linear_model import LinearRegression
lin = LinearRegression()
log_y = np.log(y)
lin.fit(X_train, log_y) # 모델 훈련

LinearRegression()

### 모델 성능 검증
훈련을 마친 상태이니 예측을 해본 후 RMSLE값까지 확인해보겠습니다.

In [6]:
# 현재 따로 분리된 테스트셋이나, validation set이 없는 관계로 임시로 X_train 데이터셋을 넣어 예측합니다.
preds = lin.predict(X_train)

# log_y(타겟값)와 예측결과인 preds 사이의 RMSLE값을 구해 모델 성능을 평가합니다.
print(f"선형 회귀의 RMSLE값 : {round(rmsle(log_y, preds), 4)}")

선형 회귀의 RMSLE값 : 1.2056


### 모델 성능 개선

기본적인 과정을 모두 거쳐 모델의 성능을 확인했을 때 그 결과가 별로라면, 다시 피처 엔지니어링으로 돌아가 데이터를 추가 분석합니다. 이후 과정들을 반복하면서 모델의 성능을 높이는게 일반적이지만, 지금의 경우 연습용으로 만드는 모델이기 때문에 여러 모델들을 변경하면서 더 나은 성능을 보이는 모델을 채택합니다.

### Ridge Regression, GridSearchCV
릿지 모델은 L2 규제를 적용한 선형 회귀 모델입니다. 하지만 기본 단순 선형 회귀 모델보다 성능이 낮다고 알려져 잘 사용하지는 않습니다. 모델의 종류 중 하나로 릿지라는 모델이 있구나 라는 정도로만 인식하는 것이 좋겠습니다.

GridSearch의 경우 모델의 하이퍼 파라미터를 격자처럼 촘촘하게 순회하며 최적의 하이퍼 파라미터 값을 찾는 기법입니다. 각 하이퍼 파라미터를 적용한 모델마다 교차 검증하며 성능을 측정하여 최종적으로 성능이 가장 좋았을 때의 하이퍼 파라미터 값을 찾아줍니다.

In [7]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge = Ridge()

# 그리드 서치 객체에 필요한 세 가지 속성이 있습니다.
# 1. 비교 검증해볼 하이퍼 파라미터값 목록
# 2. 개선하려는 모델
# 3. 교차 검증용 평가 수단(평가 함수)
# 릿지 모델에서 중요한 하이퍼 파라미터는 alpha 값으로, 값이 클수록 규제 강도가 강해집니다. 적절한 규제를 적용한다면 과대적합 문제를 개선할 수 있습니다.
ridge_params = {"max_iter":[3000],
                "alpha":[0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000]}

# 교차 검증용 평가함수
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드 서치 객체 생성
gs_ridge = GridSearchCV(
    estimator=ridge, # 릿지 모델
    param_grid=ridge_params, # 값 목록
    scoring=rmsle_scorer, # 평가 지표
    cv=5 # 교차 검증 분할 수
)

In [8]:
# grid search 실행
log_y = np.log(y)
gs_ridge.fit(X_train, log_y)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400,
                                   800, 900, 1000],
                         'max_iter': [3000]},
             scoring=make_scorer(rmsle, greater_is_better=False))

In [9]:
# 어떤 값이 최적의 하이퍼 파라미터로 선정되었는지 확인합니다.
print(f"최적의 하이퍼 파라미터 : {gs_ridge.best_params_}")

최적의 하이퍼 파라미터 : {'alpha': 0.1, 'max_iter': 3000}


In [10]:
# 가장 훈련이 잘 된 모델의 경우 gs_ridge.best_estimator_ 에 저장되어 있기 때문에 예측은 저걸로 진행하면 됩니다.
preds = gs_ridge.best_estimator_.predict(X_train)
# 평가
print(f"릿지 회귀 RMSLE값 : {round(rmsle(log_y, preds), 4)}")

# 릿지 회귀 모델의 경우 딱히 큰 성능에 대한 변화를 느끼지 못했습니다. 다음 모델을 살펴보겠습니다.

릿지 회귀 RMSLE값 : 1.2056


### Lasso
라쏘 모델은 L1 규제를 적용한 선형 회귀 모델입니다.

In [11]:
# alpha : 규제 강도를 조절하는 파라미터
from sklearn.linear_model import Lasso

# 모델 생성
lasso = Lasso()

# 하이퍼 파라미터 값 목록
lasso_alpha = 1/np.array([0.1, 1, 2, 3, 4, 10, 30, 100, 200, 300, 400, 800, 900, 1000])
lasso_params = {"max_iter":[3000], 'alpha':lasso_alpha}

# 그리드 서치 객체 생성
grid_lasso = GridSearchCV(estimator=lasso, param_grid=lasso_params, scoring=rmsle_scorer, cv=5)

# 그리드 서치 수행
log_y = np.log(y)
grid_lasso.fit(X_train, log_y)
print(f"최적 하이퍼 파라미터 : {grid_lasso.best_params_}")

최적 하이퍼 파라미터 : {'alpha': 0.001, 'max_iter': 3000}


### Lasso 성능 검증

In [12]:
# 예측
preds = grid_lasso.best_estimator_.predict(X_train)

# 평가
print(f"라쏘 회귀 RMSLE 값 : {round(rmsle(log_y, preds), 4)}")

라쏘 회귀 RMSLE 값 : 1.2056
